In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/toxic_data/train.csv")
print(df.head())

In [ ]:
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

df["label"] = (df[label_cols].sum(axis=1) > 0).astype(int)
df_binary = df[["comment_text", "label"]]

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_binary)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["comment_text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["comment_text"])

In [ ]:
from transformers import BertForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("/content/drive/MyDrive/models/lora-toxic-bert")
tokenizer.save_pretrained("/content/drive/MyDrive/models/lora-toxic-bert")